In [28]:
import os
import numpy as np
import tensorflow as tf
import trainer_2d
import data_utils

In [29]:
import importlib
importlib.reload(trainer_2d)
importlib.reload(data_utils)

<module 'data_utils' from '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/src/data_utils.py'>

In [30]:
import matplotlib.pyplot as plt
import pandas as pd

from tqdm import tqdm

Process data

In [34]:
import pickle

with open('/Users/nastya/Documents/ucu/thesis/data/triangles_split/test_triangles.pkl', "rb") as f:
    test_set = pickle.load(f)

In [35]:
test_set = [((pair[1] > 127).view(np.uint8) * 255, (pair[0] > 127).view(np.uint8) * 255) for pair in test_set]

Create metrics dataframe

In [36]:
metrics_data = pd.DataFrame(columns=['image_pair', 'mse', 'dice'])

Demons

In [37]:
model_path = '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/src/triangles/model'
general_prediction_path = '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/src/triangles/prediction'

for idx, (source_image, target_image) in tqdm(enumerate(test_set)):

    prediction_path = os.path.join(general_prediction_path, f'pair_{idx}')

    source_image = trainer_2d._load_data_from_array(source_image)
    target_image = trainer_2d._load_data_from_array(target_image)
    
    source_label = trainer_2d._one_hot_label(source_image, labels=(0, 255))
    target_label = trainer_2d._one_hot_label(target_image, labels=(0, 255))

    training_data = {'target_image': target_image, 'target_label': target_label,
                     'source_image': source_image, 'source_label': source_label,
                     'target_affine': np.eye(4), 'target_header': None
                    }
    
    with tf.Graph().as_default(), tf.device('/cpu:0'):
        trainer = trainer_2d.Trainer(input_size=(source_image.shape[1], source_image.shape[2]), num_channels=1, num_classes=2,
                          demons_type='diffeomorphic', demons_force='moving', normalization='z-score',
                          max_length=2., exp_steps=7)

        metrics, save_path = trainer.train(training_data, training_iters=100,
                                           display_steps=100,
                                           model_path=model_path,
                                           prediction_path=prediction_path)

        try:
            new_row = {'image_pair': idx, 'mse': trainer.final_mse, 'dice': trainer.final_dice}
        except:
            new_row = {'image_pair': idx, 'mse': None, 'dice': None}
        
        metrics_data.loc[len(metrics_data)] = new_row

0it [00:00, ?it/s]2024-04-25 14:17:14,137 Removing '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/src/triangles/prediction/pair_0'
2024-04-25 14:17:14,138 Removing '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/src/triangles/model'
2024-04-25 14:17:14,139 Allocating '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/src/triangles/prediction/pair_0'
2024-04-25 14:17:14,140 Allocating '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/src/triangles/model'
2024-04-25 14:17:14,247 Start optimization with demons configuration: demons force: moving, demons type: diffeomorphic, maximum step length: 2.0 pixels, exponential steps: 7.0
2024-04-25 14:17:14,469 [Iteration 0], [Metrics] MSE= 0.1050, Dice= 0.7887
2024-04-25 14:17:14,552 [Iteration 1], [Metrics] MSE= 0.0947, Dice= 0.7887
2024-04-25 14:17:14,558 [Iteration 2], [Metrics] MSE= 0.0864, Dice= 0.8160
2024-04-25 14:17:14,563 [Iteration 3], [Metrics] MSE= 0.0778, Dice= 0.8330
2024-04-25 14:17:14,569 [Iteration

In [38]:
metrics_data

,image_pair,mse,dice
0,0,0.009819,0.978373
1,1,0.004621,0.982563
2,2,0.096229,0.842027
3,3,0.007111,0.971479
4,4,0.006713,0.990215
...,...,...,...
195,195,0.019906,0.960271
196,196,0.009273,0.911380
197,197,0.008911,0.982345
198,198,0.001736,0.986551


In [39]:
metrics_data['dice'].mean()

0.8728463

In [40]:
metrics_data['mse'].mean()

0.038435582

In [43]:
metrics_data.to_csv('/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/src/triangles/metrics.csv')